In [ ]:
!pip install tensorflow
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets

### Importación de librerías

In [1]:
from typing import Dict, Text

import numpy as np
import tensorflow as tf

import tensorflow_recommenders as tfrs

2022-04-13 22:39:27.272719: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-13 22:39:27.272736: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
import pandas as pd
import re

### Variables generales

In [3]:
pathToDF = "../../Inputs/Creados - Proyecto/"
fileToDF = "dfVentasDefinitivo.csv"

### Importación de datos

In [4]:
df = pd.read_csv(f"{pathToDF}{fileToDF}")

In [5]:
#Sólo ejecutar una vez, elimina primera columna si la exportación de dicho CSV no se realizó con el parámetro "index=False"
df.drop(columns=df.columns[0], axis=1, inplace=True)

### Depuración de datos

In [6]:
df["name"] = df["name"].apply(lambda x: re.sub("\d+\s*\S*\w+\s*\S*\w", "", x))

In [7]:
df2 = df[["customer_id", "name"]]

### Aplicación de TensorFlow

In [8]:
ratings = tf.data.Dataset.from_tensor_slices(dict(df2))

2022-04-13 22:41:23.141929: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-13 22:41:23.141961: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-13 22:41:23.141977: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (c19b3611085a): /proc/driver/nvidia/version does not exist
2022-04-13 22:41:23.150703: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


KeyboardInterrupt: 

In [ ]:
for row in ratings.take(3):
  print(row)

In [ ]:
# Select the basic features.
ratings = ratings.map(lambda x: {
    "customer_id": x["customer_id"],
    "name": x["name"]
})
movies = ratings.map(lambda x: x["name"])

In [ ]:
user_ids_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
user_ids_vocabulary.adapt(ratings.map(lambda x: x["customer_id"]))

movie_titles_vocabulary = tf.keras.layers.StringLookup(mask_token=None)
movie_titles_vocabulary.adapt(movies)

In [ ]:
class MovieLensModel(tfrs.Model):
  # We derive from a custom base class to help reduce boilerplate. Under the hood,
  # these are still plain Keras Models.

  def __init__(
      self,
      user_model: tf.keras.Model,
      movie_model: tf.keras.Model,
      task: tfrs.tasks.Retrieval):
    super().__init__()

    # Set up user and movie representations.
    self.user_model = user_model
    self.movie_model = movie_model

    # Set up a retrieval task.
    self.task = task

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    # Define how the loss is computed.

    user_embeddings = self.user_model(features["customer_id"])
    movie_embeddings = self.movie_model(features["name"])

    return self.task(user_embeddings, movie_embeddings)

In [ ]:
# Define user and movie models.
user_model = tf.keras.Sequential([
    user_ids_vocabulary,
    tf.keras.layers.Embedding(user_ids_vocabulary.vocab_size(), 64)
])
movie_model = tf.keras.Sequential([
    movie_titles_vocabulary,
    tf.keras.layers.Embedding(movie_titles_vocabulary.vocab_size(), 64)
])

# Define your objectives.
task = tfrs.tasks.Retrieval(metrics=tfrs.metrics.FactorizedTopK(
    movies.batch(128).map(movie_model)
  )
)

In [ ]:
# Create a retrieval model.
model = MovieLensModel(user_model, movie_model, task)
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.5))

# Train for 3 epochs.
model.fit(ratings.batch(4096), epochs=3)

# Use brute-force search to set up retrieval using the trained representations.
index = tfrs.layers.factorized_top_k.BruteForce(model.user_model)
index.index_from_dataset(
    movies.batch(100).map(lambda title: (title, model.movie_model(title))))

In [ ]:
# Get some recommendations.
_, titles = index(np.array(["42"]))
print(f"Top 3 recommendations for user 42: {titles[0, :3]}")

In [ ]:
model.save_weights(".")

In [ ]:
model.save_weights('FirstRecommenderModel.tf')